# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

In [7]:
hook = sy.TorchHook(th)

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [9]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [11]:
x = th.tensor([1,2,3,4,5])

In [12]:
#send pointer back to the remote tensor. w
#we can treat it as a normal pytorch tensor because it is wrapped this way. And it has the full tensor API.
#however instead of executing those commands locally, It seirliazes the command and send it to bob then bob 
# execute on our behaf and returns the pointer to the new tensor.
x = x.send(bob)

In [13]:
bob._objects

{74564530215: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

74564530215

In [16]:
x.id

95146940444

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
#This worker was created when we imported an hooked pysyft
#whenever we communicate with a remote machine, what we are really doing whenever we execute a command 
#regarding x(or any otheer tensor at bob), Hey local worker contact bob and tell him to do this.
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:95146940444 -> bob:74564530215]

In [20]:
#get the information back from bob
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
#now bob does not have tensors because we got back the only tensor he had
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [22]:
# try this project here!
bob = sy.VirtualWorker(hook, id="bob")
alice= sy.VirtualWorker(hook, id="alice")

In [23]:
x = th.tensor([1,2,3,4,5])
#The bellow pointer is a multi pointer which is a pointer that points to multiple machines
x_ptr = x.send(bob,alice)

In [24]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:62011627709 -> bob:3123695506]
	-> [PointerTensor | me:91912028006 -> alice:93580722200]

In [25]:
#returns a dictionary to multiple pointers
x_ptr.child.child

{'bob': [PointerTensor | me:62011627709 -> bob:3123695506],
 'alice': [PointerTensor | me:91912028006 -> alice:93580722200]}

In [26]:
#Returns two objects instead of one because what was contained on this variable was two pointer objects
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [27]:
x = th.tensor([1,2,3,4,5])
x_ptr = x.send(bob,alice)
#We can have it automatically sum the tensors then return them
x_ptr.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [38]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [39]:
#poiner tensor pointer is located on a worker called me which is the local_worker
#and it is pointing at a tensor on bob's machine with an ID of 117422....
x

(Wrapper)>[PointerTensor | me:21701934682 -> bob:80652398810]

In [40]:
y

(Wrapper)>[PointerTensor | me:32512235696 -> bob:12667487899]

In [41]:
#return a pointer to the output of this operation that happened on bob's machine 
z = x + y

In [42]:
z

(Wrapper)>[PointerTensor | me:8182332322 -> bob:82947348097]

In [43]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [44]:
#same as z= x+y and will return a pointer to the output again
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:44685607659 -> bob:94421717477]

In [45]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [46]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [47]:
z = (x + y).sum()

In [48]:
z.backward()

(Wrapper)>[PointerTensor | me:53545254472 -> bob:22623290764]

In [49]:
x = x.get()

In [50]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [51]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [55]:
# try this project here!
input_x = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.] ,[1],[0],[0]] , requires_grad=True).send(bob)

In [56]:
weights=th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [57]:
for i in range(10):
    pred=input_x.mm(weights)
    loss=((pred-target)**2).sum()
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *=0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [67]:
bob = bob.clear_objects()

In [68]:
bob._objects

{}

In [69]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [70]:
bob._objects

{23258641330: tensor([1, 2, 3, 4, 5])}

In [71]:
#when we delete the pointer that is pointing to the remote object, The object gets deleted as well
#This is controlled by syft attribute called "garbage_collect_data" which is set to True by default
del x

In [65]:
bob._objects

{44014706865: tensor([1, 2, 3, 4, 5])}

In [72]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [73]:
bob._objects

{25803690210: tensor([1, 2, 3, 4, 5])}

In [75]:
#when the pointer gets garbage collected, I send a message to bob saying hey delete that tensor
x.child.garbage_collect_data

True

In [76]:
x = "asdf"

In [77]:
bob._objects

{}

In [78]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [79]:
#In jupyter notebook If we call this the garbage collection does not work.
#because when we excute this display command another pointer points to the same object so when we delete one, 
#It doesn't send a message to bob to delete it.
x

(Wrapper)>[PointerTensor | me:52581403300 -> bob:3003579215]

In [80]:
bob._objects

{3003579215: tensor([1, 2, 3, 4, 5])}

In [81]:
x = "asdf"

In [82]:
bob._objects

{3003579215: tensor([1, 2, 3, 4, 5])}

In [83]:
del x

In [84]:
bob._objects

{3003579215: tensor([1, 2, 3, 4, 5])}

In [85]:
bob = bob.clear_objects()
bob._objects

{}

In [88]:
#if we send 100 tensors to bob but return it to the same pointer, 
# Then bob deletes the tensor that was pointed to by x and makes a new tensor and return it's pointer
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [89]:
bob._objects

{10489851967: tensor([1, 2, 3, 4, 5])}

In [90]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [91]:
#gives back an error because we are trying to add a pointer tensor to a tensor,
#because the two tensors are on different machines it gives an error
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:61291734512 -> bob:40194041875]
Tensor B: tensor([1, 1, 1, 1, 1])

In [92]:
#Same error because the two tensors are on two different machiens
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [93]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [94]:
from torch import nn, optim

In [95]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [96]:
# A Toy Model
model = nn.Linear(2,1)

In [97]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [98]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.0040)
tensor(0.3765)
tensor(0.1771)
tensor(0.1119)
tensor(0.0741)
tensor(0.0496)
tensor(0.0334)
tensor(0.0227)
tensor(0.0155)
tensor(0.0107)
tensor(0.0074)
tensor(0.0052)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0013)
tensor(0.0010)
tensor(0.0007)
tensor(0.0005)
tensor(0.0004)


In [99]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [100]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [101]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [102]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [103]:
train()

tensor(1.0733, requires_grad=True)
tensor(0.8801, requires_grad=True)
tensor(0.2286, requires_grad=True)
tensor(0.4114, requires_grad=True)
tensor(0.1724, requires_grad=True)
tensor(0.2356, requires_grad=True)
tensor(0.1014, requires_grad=True)
tensor(0.1371, requires_grad=True)
tensor(0.0587, requires_grad=True)
tensor(0.0799, requires_grad=True)
tensor(0.0339, requires_grad=True)
tensor(0.0467, requires_grad=True)
tensor(0.0196, requires_grad=True)
tensor(0.0273, requires_grad=True)
tensor(0.0113, requires_grad=True)
tensor(0.0160, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0094, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0055, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0003, requi

In [ ]:
# when we have our model after and before the training we can still reverse engineer what the data was.
# solutions:
# 1- train multiple different models in parrallel on different workers on different people's dataset
# then average those models together, then the only model we see and get back is an average of multiple models. 

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [128]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [129]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [130]:
x = x.send(alice)

In [131]:
bob._objects

{49261783994: tensor([1, 2, 3, 4, 5])}

In [132]:
#alice has a pointer pointing to bob's tensor
alice._objects

{66748941998: (Wrapper)>[PointerTensor | alice:66748941998 -> bob:49261783994]}

In [133]:
y = x + x

In [134]:
y

(Wrapper)>[PointerTensor | me:54722118061 -> alice:52740722316]

In [135]:
#we have two pointers in bob's machine y and x
bob._objects

{49261783994: tensor([1, 2, 3, 4, 5]),
 20678589506: tensor([ 2,  4,  6,  8, 10])}

In [136]:
# we have two pointers on alice's machine to bob's machine
# the new pointer (Y) that was created. has the exact same pointer chain (ownership chain) as the original tensor
# that was used to create it
alice._objects

{66748941998: (Wrapper)>[PointerTensor | alice:66748941998 -> bob:49261783994],
 52740722316: (Wrapper)>[PointerTensor | alice:52740722316 -> bob:20678589506]}

In [137]:
jon = sy.VirtualWorker(hook, id="jon")

In [138]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [139]:
# Because x and y does not have the same pointer chain or ownership structure It will trigger an error
z = x+y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:alice #objects:1> while the other is on <VirtualWorker id:jon #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [140]:
bob._objects

{86796976004: tensor([1, 2, 3, 4, 5]), 28623529026: tensor([1, 2, 3, 4, 5])}

In [141]:
alice._objects

{93474357842: (Wrapper)>[PointerTensor | alice:93474357842 -> bob:86796976004]}

In [142]:
#we are now pointing to bob's data directly and alice does not have the pointer anymore
x = x.get()
x

(Wrapper)>[PointerTensor | me:93474357842 -> bob:86796976004]

In [143]:
bob._objects

{86796976004: tensor([1, 2, 3, 4, 5]), 28623529026: tensor([1, 2, 3, 4, 5])}

In [144]:
alice._objects

{}

In [145]:
# now we get bob's tensor and thus bob will not have this tensor in it
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [150]:
bob._objects

{8254705553: tensor([1, 2, 3, 4, 5])}

In [162]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [163]:
bob._objects

{78118535176: tensor([1, 2, 3, 4, 5])}

In [164]:
alice._objects

{44787372382: (Wrapper)>[PointerTensor | alice:44787372382 -> bob:78118535176]}

In [165]:
# Garbage collection will collect the whole chain removing the tensor from both bob and alice when the pointer is deleted
del x

In [166]:
bob._objects

{}

In [159]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [176]:
bob.clear_objects()
alice.clear_objects()
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [177]:
bob._objects

{61238474655: tensor([1, 2, 3, 4, 5])}

In [178]:
alice._objects

{89994512826: (Wrapper)>[PointerTensor | alice:89994512826 -> bob:61238474655]}

In [179]:
#moves the tensor from bob to alice, without touching our machine
x.remote_get()

(Wrapper)>[PointerTensor | me:14419842004 -> alice:89994512826]

In [180]:
bob._objects

{}

In [181]:
alice._objects

{89994512826: tensor([1, 2, 3, 4, 5])}

In [197]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [198]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [199]:
bob._objects

{23655871200: tensor([1, 2, 3, 4, 5])}

In [200]:
alice._objects

{}

In [201]:
# Move tensor from bob to alice witout touching our pc. by using remote get under the hood
x.move(alice)

(Wrapper)>[PointerTensor | me:9238476529 -> alice:6881432265]

In [202]:
bob._objects

{}

In [203]:
alice._objects

{6881432265: tensor([1, 2, 3, 4, 5])}